In [1]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn import metrics

In [3]:
df = pd.read_csv('data.csv')

#df.shape = (24210, 27)

In [4]:
df.columns

Index(['Unnamed: 0', 'index', 'title', 'company_name', 'location', 'via',
       'description', 'extensions', 'job_id', 'thumbnail', 'posted_at',
       'schedule_type', 'work_from_home', 'salary', 'search_term', 'date_time',
       'search_location', 'commute_time', 'salary_pay', 'salary_rate',
       'salary_avg', 'salary_min', 'salary_max', 'salary_hourly',
       'salary_yearly', 'salary_standardized', 'description_tokens'],
      dtype='object')

In [5]:
df.head()

,Unnamed: 0,index,title,company_name,location,via,description,extensions,job_id,thumbnail,...,commute_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,0,Data Analyst,Robert Half,"Oklahoma City, OK",via LinkedIn,Description\n\nRobert Half is looking for a pr...,"['24 hours ago', 'Contractor', 'Health insuran...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['go']
1,1,1,Data Analyst,Apex Health Solutions,United States,via LinkedIn,Data Analyst Summary Apex Health Solutions is ...,"['21 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'power_bi', 'azure', 'sql']"
2,2,2,Marketing Data Analyst,Ledger Bennett,Anywhere,via LinkedIn,"At Ledger Bennett, we strive to help our emplo...","['21 hours ago', 'Work from home', 'Full-time']",eyJqb2JfdGl0bGUiOiJNYXJrZXRpbmcgRGF0YSBBbmFseX...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'tableau', 'sql']"
3,3,3,"Boolean Search Data Analyst | $54,000-$108,000...",IT Pros,Anywhere,via Indeed,Company Description\n\nJoin a world-class data...,"['14 hours ago', 'Work from home', 'Part-time'...",eyJqb2JfdGl0bGUiOiJCb29sZWFuIFNlYXJjaCBEYXRhIE...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,4,4,Product Data Analyst - Ditch Witch - Now Hiring,The Toro Company,"Perry, OK",via Snagajob,Who Are We? \n\nOur plant in West Salem sta...,"['22 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJQcm9kdWN0IERhdGEgQW5hbHlzdC...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'sap', 'assembly']"


In [6]:
df = df[['title', 'company_name', 'location', 'via', 'description', 'extensions','salary_standardized', 'description_tokens']]

In [13]:
text = df['description']

In [95]:
print(df['description'][120])

Sogeti is looking for a talented professional Consultant to join our Colorado team built on the success of our people and partnerships. Come be a part of our “People centric” culture!

Sogeti is a wholly-owned subsidiary of Capgemini and the Technology and Engineering Services Division of Capgemini. We have more than 25,000 professionals in 15 countries and has a strong local presence in over 100... locations scattered throughout the US, Europe and India.

We are leading provider of technology and software testing, specializing in Application, Infrastructure, Data, Analytics and Engineering Services. We offer cutting-edge solutions around Testing, Business Intelligence & Analytics, PLM, Mobile, Cloud, and Cyber Security, combining world class methodologies and its global delivery model, Rightshore®. Sogeti’s roots and the bedrock of our success lie in the provision of local professional services - teams of experienced IT professionals working closely with local clients.

We offer an en

In [19]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Remove newlines, tabs, and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    return text.strip()


In [21]:
text = text.apply(preprocess_text)

In [23]:
#Singlets

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')

tfidf_matrix = vectorizer.fit_transform(text)

feature_names = vectorizer.get_feature_names_out()

def count_words(tfidf_matrix, feature_names, min_count=100):
    word_counter = Counter()
    
    for row in range(tfidf_matrix.shape[0]):
        row_data = tfidf_matrix[row].toarray().flatten()
        
        for idx, score in enumerate(row_data):
            if score > 0:
                word_counter[feature_names[idx]] += 1

    sorted_words = [(word, count) for word, count in word_counter.items() if count > min_count]
    sorted_words = sorted(sorted_words, key=lambda x: x[1], reverse=True)
    
    return sorted_words

sorted_words = count_words(tfidf_matrix, feature_names, min_count=100)

for word, count in sorted_words[:100]: 
    print(f"{word}: {count}")

data: 22547
experience: 19989
work: 18088
business: 16995
skills: 16281
analysis: 15957
years: 15733
team: 14601
analyst: 14530
degree: 14049
support: 13211
ability: 12995
including: 12661
analytics: 12550
management: 12507
requirements: 12403
sql: 12359
information: 12286
working: 12144
tools: 12117
strong: 11908
related: 11907
required: 11849
knowledge: 11790
role: 11438
job: 11105
solutions: 10996
develop: 10973
development: 10819
responsibilities: 10816
provide: 10811
preferred: 10776
using: 10728
position: 10452
new: 10414
opportunity: 10385
reporting: 10227
reports: 10215
communication: 10015
analytical: 9935
time: 9479
complex: 9465
process: 9402
identify: 9400
technical: 9323
company: 9317
benefits: 9258
processes: 9222
projects: 9205
needs: 9167
help: 9144
teams: 9119
systems: 9107
science: 9043
environment: 9028
disability: 8849
equal: 8800
status: 8794
qualifications: 8734
multiple: 8706
opportunities: 8591
national: 8514
field: 8496
insights: 8477
analyze: 8449
employer: 84

In [25]:
len(sorted_words)

4342

In [41]:
trimmed_tupples = [tup for tup in sorted_words if tup[1] > 500]

In [43]:
len(trimmed_tupples)

1750

In [55]:
with open("singlets_Output.txt", "w") as text_file:
    text_file.write(str(trimmed_tupples))

In [ ]:
#Bigrams

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

vectorizer = TfidfVectorizer(ngram_range=(2, 2), stop_words='english')

tfidf_matrix = vectorizer.fit_transform(text)

feature_names = vectorizer.get_feature_names_out()

def count_bigrams(tfidf_matrix, feature_names):
    bigram_counter = Counter()
    
    for row in range(tfidf_matrix.shape[0]):
        row_data = tfidf_matrix[row].toarray().flatten()
        
        for idx, score in enumerate(row_data):
            if score > 0:
                bigram_counter[feature_names[idx]] += 1
 
    return sorted_bigrams

sorted_bigrams = count_bigrams(tfidf_matrix, feature_names)

In [ ]:
for bigram, count in sorted_bigrams[:100]: 
    print(f"{bigram}: {count}")